# **CSC 4008 Assignment 3 (Q1.a) - 119020062 许文仲**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/drive/My Drive')

Mounted at /content/gdrive


In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [ ]:
# spark = SparkSession.builder.getOrCreate()
sc = pyspark.SparkContext.getOrCreate()
# path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/test.txt"
real_path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/graph-full.txt"
check_path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/graph-small.txt"

data = sc.textFile(real_path)
# rdd.collect()

In [ ]:
# get the first level friend RDD
size = 1000
rdd = data.map(lambda x: x.split("\t"))
rdd2 = rdd.map(lambda x: (int(x[0]), int(x[1])))
rdd_dis = rdd2.distinct()
rdd3 = rdd_dis.groupByKey().map(lambda x : (x[0], list(x[1])))
rdd4 = rdd3.sortByKey()
rdd4.collect()

In [ ]:
beta = 0.8
def findM(line):
  index = line[1]
  d_i = 1 / len(index)
  # print(d_i)
  M_i = np.zeros(shape = (1, size))[0]
  for i in index:
    M_i[i-1] = d_i
  return (np.array(M_i))

M_matrix = rdd4.map(findM)
z = np.array(M_matrix.collect())

In [ ]:
M = z.T
def test(line):
  index = line[0]
  return (index, M[index-1])
M_matrix = rdd4.map(test)

In [ ]:
r_odd = np.full(size, 1/size)
def find_r_new(line):
  M_i = line[1]
  result = beta*np.matmul(r_odd, M_i) + (1-beta)/size
  return (result)

temp = M_matrix.map(find_r_new)
r_new = np.array(temp.collect())
r_new

iteration = 1
while iteration <= 40:
  r_odd = r_new
  temp = M_matrix.map(find_r_new)
  r_new = np.array(temp.collect())
  iteration += 1
r_new

In [ ]:
k = 5  #前k个
high = r_new.argsort()[-k:][::-1] + 1 #获取前k个索引
low = r_new.argsort()[:k] + 1
highest_score = []
lowest_score = []
for i in range(0,k):
  highest_score.append(r_new[high[i]-1])
  lowest_score.append(r_new[low[i]-1])

print("The first 5 highest index of PageRank are: ", high)
print("The corresponding scores are: ", highest_score)
print("The first 5 lowest index of PageRank are: ", low)
print("The corresponding scores are: ", lowest_score)

The first 5 highest index of PageRank are:  [263 537 965 243 285]
The corresponding scores are:  [0.0020202911815182184, 0.0019433415714531497, 0.0019254478071662631, 0.001852634016241731, 0.0018273721700645144]
The first 5 lowest index of PageRank are:  [558  93  62 424 408]
The corresponding scores are:  [0.0003286018525215297, 0.0003513568937516577, 0.00035314810510596274, 0.0003548153864930145, 0.00038779848719291705]
